In [2]:
import pandas as pd
import requests
import time
import random
import html
import logging
from tqdm import tqdm


In [3]:
# =======================
# 1. CARGAR Y LIMPIAR CSV
# =======================
# Función para limpiar nombres
import pandas as pd
import re

def limpiar_nombre(nombre):
    if pd.isna(nombre) or not nombre.strip():
        return None
    # Eliminar caracteres problemáticos para SPARQL
    nombre = nombre.replace("\\", "")   # Quitar backslashes
    nombre = nombre.replace('"', '')    # Quitar comillas dobles
    nombre = nombre.replace("'", "")    # Quitar comillas simples
    nombre = nombre.replace(",", "")    # Quitar comas
    nombre = nombre.replace("/", " ")   # Evitar errores con `/`
    nombre = nombre.replace("&", "and") # Evitar ambigüedad
    nombre = nombre.strip()
    return nombre

# Cargar y limpiar
df = pd.read_csv("../data/artists.csv", header=None, names=["raw"])
nombres_limpios = [limpiar_nombre(nombre) for nombre in df["raw"]]
artistas_unicos = sorted(set([nombre for nombre in nombres_limpios if nombre]))

print(f"✅ Total artistas únicos: {len(artistas_unicos)}")


# =======================
# 2. CONSULTA SPARQL CON TODOS LOS CAMPOS
# =======================
WIKIDATA_ENDPOINT = "https://query.wikidata.org/sparql"
HEADERS = {
    "Accept": "application/sparql-results+json",
    "User-Agent": "Workshop/1.0 (ejemplo@gmail.com)"  # <-- ¡Pon tu info real aquí!
}


def construir_query_sparql(artistas):
    values = "\n".join([f'"{nombre}"@en' for nombre in artistas])
    query = f"""
    SELECT ?artistLabel ?death ?countryLabel ?awardLabel ?genderLabel WHERE {{
      VALUES ?name {{ {values} }}
      ?artist rdfs:label ?name.
      ?artist wdt:P166 ?award.
      ?award rdfs:label ?awardLabel.
      OPTIONAL {{ ?artist wdt:P27 ?country. }}
      OPTIONAL {{ ?artist wdt:P570 ?death. }}
      OPTIONAL {{ ?artist wdt:P21 ?gender. }}  # Género con P21
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
    }}
    """
    return query

def obtener_datos_wikidata(artistas_batch):
    query = construir_query_sparql(artistas_batch)
    try:
        response = requests.post(WIKIDATA_ENDPOINT, data={"query": query}, headers=HEADERS)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print("❌ Error en SPARQL:", e)
        return None

# =======================
# 3. CONSULTA ROBUSTA
# =======================
print("🚀 Consultando Wikidata...")

MAX_QUERY_SIZE = 60000
results = []
i = 0

with tqdm(total=len(artistas_unicos), desc="🔎 Batches SPARQL") as pbar:
    while i < len(artistas_unicos):
        batch_size = 80
        batch_success = False

        while batch_size > 0 and not batch_success:
            batch = artistas_unicos[i:i+batch_size]
            query = construir_query_sparql(batch)
            if len(query.encode("utf-8")) > MAX_QUERY_SIZE:
                batch_size -= 5
                continue

            data = obtener_datos_wikidata(batch)
            if data:
                for row in data["results"]["bindings"]:
                    results.append({
                        "artist": row.get("artistLabel", {}).get("value", ""),
                        "death": row.get("death", {}).get("value", ""),
                        "country": row.get("countryLabel", {}).get("value", ""),
                        "award": row.get("awardLabel", {}).get("value", "No awards"),
                        "gender": row.get("genderLabel", {}).get("value", "Unknown")  # Valor por defecto si no hay género
                    })
                batch_success = True
                i += batch_size
                pbar.update(batch_size)
                time.sleep(0.8)
            else:
                batch_size = batch_size // 2

        if not batch_success:
            print(f"⚠️  Saltando artista en índice {i}: {artistas_unicos[i]}")
            i += 1
            pbar.update(1)

# =======================
# 4. GUARDAR RESULTADOS
# =======================
columnas_ordenadas = ["artist", "country", "award", "death", "gender"]
df_result = pd.DataFrame(results, columns=columnas_ordenadas)

✅ Total artistas únicos: 31437
🚀 Consultando Wikidata...


🔎 Batches SPARQL: 31440it [12:15, 42.77it/s]                           


In [4]:
# Calcular el punto de división (por la mitad en este caso)
halfway = len(df_result) // 2

# Guardar la primera parte del DataFrame
df_result.iloc[:halfway].to_csv("../data/wikidata_artists_part1.csv", index=False)
print("✅ Archivo 'wikidata_artists_part1.csv' creado correctamente.")

# Guardar la segunda parte del DataFrame
df_result.iloc[halfway:].to_csv("../data/wikidata_artists_part2.csv", index=False)
print("✅ Archivo 'wikidata_artists_part2.csv' creado correctamente.")

✅ Archivo 'wikidata_artists_part1.csv' creado correctamente.
✅ Archivo 'wikidata_artists_part2.csv' creado correctamente.
